<a href="https://colab.research.google.com/github/Ahtesham519/Praticle_nlp_book/blob/main/CNN_RNN_ATIS_intents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-preprocessing


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00


In [2]:
#gerneral imports
import os
import sys
import random
random.seed(0)

#basicimports
import numpy as np
import pandas as pd

#NN imports
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense , Input , GlobalMaxPooling1D
from keras.layers import Conv1D , MaxPooling1D , Embedding , LSTM
from keras.models import Model, Sequential
from keras.initializers import Constant

#Encoder
from sklearn.preprocessing import LabelEncoder


In [17]:
!pip install data 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from google.colab import files

# Upload the data_utils.py file to Colab
uploaded = files.upload()

# Read the content of the data_utils.py file
file_content = uploaded['data_utils.py'].decode('utf-8')




Saving data_utils.py to data_utils (1).py


In [27]:
from google.colab import files
import importlib.util

# Upload the data_utils.py file to Colab
uploaded = files.upload()

# Get the path of the uploaded data_utils.py file
file_path = next(iter(uploaded))

# Load the data_utils module from the uploaded file
spec = importlib.util.spec_from_file_location("data_utils", file_path)
data_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(data_utils)

# Import the fetch_data and read_method functions
fetch_data = data_utils.fetch_data
read_method = data_utils.read_method

sents, labels , intents = fetch_data('Data/data2/atis.train.w.w-intent.iob')

train_sentences = [" ".join(i) for i in sents]

train_texts = train_sentences
train_labels = intents.tolist()

vals = []

for i in range(len(train_labels)):
  if "#" in train_labels[i]:
    vals.append(i)

for i in vals[:: -1]:
  train_labels.pop(i)
  train_texts.pop(i)

print("Number of training sentences : " ,len(train_texts))
print("number of unique intents : " , len(set(train_labels)))

for i in zip(train_texts[:5] , train_labels[:5]):
  print(i)

Saving data_utils.py to data_utils.py


AttributeError: ignored

Data Preprocesssing

In [28]:
MAX_SEQUENCE_LENGHT = 300
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.3

In [29]:
tokenizer = Tokenizer*(num_words = MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_squences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


SyntaxError: ignored

In [30]:
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)


NameError: ignored

In [31]:
#Converting files to sequences to be fed into the neural network . MAX seq. len is 1000 as set earlier
#initial pdding of 0s, until vector is of size MAX_SEQUENCE_LENGTH

trainvalid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences , maxlen = MAX_SEQUENCE_LENGTH)
trainvalid_labels = to_categorical(train_labels)

test_labels = to_categorical(np.asarray(test_labels) , num_classes = trainvalid_labels.shape[1])


#split the training data into a training and a validation set

indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)
trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT *trainvalid_data.shape[0])
X_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]

x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]

#this is the data we will use for CNN and RNN trainign
print('Splitting the train data into train and valid is done ')

NameError: ignored

Modeling

Embedding Matrix

In [32]:
print('Preparing embeding matrix.')

#first , build index mapping words in the embeddings set
#to their embedding vectors

#download Glove 6B from here:htt

BASE_DIR = 'Data'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR , 'glove.6B.100d.txt'), encoding = "utf-8")as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:] , dtype = 'float32')
    embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings. ' %len(embeddings_index))

num_words = min(MAX_NUM_WORDS, len(word_index)) +1
embeddings_matrix = np.zeros(num_words , EMBEDDINGS_DIM)
for word , i in word_index.items():
  if i > MAX_NUM_WORDS:
    continue
  embeddings_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embeddings_vector

embeddings_layer = Embedding(num_words,
                             EMBEDDING_DIM,
                             embeddings_initializer = Constant(embedding_matrix),
                             input_length = MAX_SEQUENCE_LENGTH,
                             trainable = False)

print("Preparing of embedding matrix is done")

Preparing embeding matrix.


FileNotFoundError: ignored

CNN with Pre-Trained Embeddings

In [33]:
print('Define a 1D CNN model .')

cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128 ,  5, activation ='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128 , 5 , activation = 'relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5 , activation = 'relu'))
cnnmodel.add(GloballMaxPooling1D())
cnnmodel.add(Dense(128 , activation = 'relu'))
cnnmodel.add(Dense(len(trainvalid_labels[0] ) activation = 'softmax'))


cnnmodel.complie(loss='categorical_crossentropy' , 
                 optimizer = 'rmsprop',
                 metrics = ['acc'])

cnnmodel.summary()

#Train the model , Tune to validation set.
cnnmodel.fit(X_train, y_train,
             batch_size =128,
             epochs =1 , 
             validation_data = (X_val , y_val))

#Evaluate on test set :
score, acc = cnnmodel.evaluate(test(test_data , test_labels))
print('Test accuracy with CNN' , acc)


SyntaxError: ignored

CNN-Embedding LAyers

In [34]:
print("Defing and training a CNN model , training embedding layer on the fly instead of using a pre-trained embeddings")

cnnmodel=Sequential()
cnnmodel.add(Embedding(MAX_NUM_WORDS , 128))
cnnmodel.add(Conv1D(128 , 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128 , 5, activation= 'relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128 , 5, activation= 'relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128 , activation = 'relu'))
cnnmodel.add(Dense(len(trainvalid_labels[0]), activation = 'softmax'))

cnnmodel.complie(loss = 'categorical_crossentropy',
                 optimizer ='rmsprop',
                 metrics =['acc'])

cnnmodel.summary()

#Train the model .Tuen to validation set
cnnmodel.fit(X_train , y_train,
             batch_size =128,
             epochs = 1 ,
             validation_data=(x_val , y_val))

#evulate on test set.
score, acc = cnnmodel.evulate(test_data , test_labels)
print('Test accuracy with CNN :' , acc)

Defing and training a CNN model , training embedding layer on the fly instead of using a pre-trained embeddings


NameError: ignored

RNN Embedding Layer

In [35]:
print("Defining and training LSTM model , training embedding layer on the fly.")

#Modified from:
rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS , 128))
rnnmodel.add(LSTM(128 , dropout = 0.2 , recurrent_dropout = 0.2))
rnnmodel.add(Dense(len(trainvalid_labels[0]) , activation = 'sigmoid'))
rnnmodel.compile(loss = 'binary_crossentropy',
                 optimizer = 'adam',
                 metrics = ['accuracy'])

rnnmodel.summary()

print('Training the RNN')
rnnmodel.fit(X_train , y_train,
             batch_size =32,
             epochs =1,
             validation_data = (x_val , y_val))
score, acc = rnnmodel.evaluate(test_data , test_labels,
                               batch_size =32)
print('Test accuracy with RNN: ' , acc)


Defining and training LSTM model , training embedding layer on the fly.


NameError: ignored

LSTM with pre-trainined Embeddings

In [36]:
print("Defining and training an LSTM model , using pre-trainined embeddings layers ")

#modified form 
rnnmodel2 = Sequential()
rnnmodel2.add(embeding_layer)
rnnmodel2.add(LSTM(128 , dropout = 0.2, recurrent_dropout = 0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]) , activation = 'sigmoid'))
rnnmodel2.compile(loss = 'binary_crossentropy' , 
                  optimizer = 'adam' ,
                  metrics = ['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(X_train , y_train,
              batch_size = 32,
              epochs = 1,
              validation_data = (x_val, y_val))
score ,acc = rnnmodel2.evulate(test_data , test_labels,
                               batch_size = 32)
print('Test accuracy with RNN:' acc)

SyntaxError: ignored